# Part 1 - Text Generation and Chat

This part focuses on text generation with the Gemini API using the `google-genai` SDK, including basic prompts, chat interactions, streaming, and configuration.

Make sure you have completed the [setup and authentication](solution_00_setup_and_authentication.md) section.

In [5]:
from google import genai
from google.genai import types
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY',None)

# Create client with api key
MODEL_ID = "gemini-2.5-flash-preview-05-20"
client = genai.Client(api_key=GEMINI_API_KEY)

## 1. Send Your First Prompt

In [6]:
prompt = "Create 3 names for a new coffee shop that emphasizes sustainability."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt
)

print("Response from Gemini:")
print(response.text)

Response from Gemini:
Here are 3 names for a new coffee shop emphasizing sustainability:

1.  **The Rooted Bean & Brew**
    *   **Why it works:** "Rooted" suggests a deep connection to the earth, ethical sourcing, and strong foundations in sustainable practices. "Bean & Brew" clearly identifies it as a coffee shop.

2.  **Light Footprint Coffee Co.**
    *   **Why it works:** "Light Footprint" is a direct and well-understood term for minimizing environmental impact, immediately communicating the shop's core value. "Coffee Co." makes it sound professional and established.

3.  **Evergreen Grind**
    *   **Why it works:** "Evergreen" evokes nature, renewal, freshness, and continuous growth – all aligned with sustainability. "Grind" is a direct, active, and memorable coffee term.


#### **!! Exercise !!**

1.  Send prompts: Write a short poem about a robot. Explain "machine learning" simply.
2.  Experiment with other models if available (e.g., `gemini-2.0-flash` or `gemini-2.0-flash-lite`).

## 2. Understanding and Counting Tokens

Tokens are the basic units that Gemini models use to process text. Understanding token usage is crucial for:
- **Cost management**: Billing is based on token consumption
- **Context limits**: Models have maximum token limits (e.g., 1M tokens for Gemini 2.5 Pro)
- **Performance optimization**: Smaller inputs generally process faster

For Gemini models, a token is equivalent to about 4 characters, and 100 tokens equals about 60-80 English words.

### Count tokens before generation

You can count tokens in your input before sending it to the model to estimate costs and ensure you stay within limits:

In [7]:
prompt = "The quick brown fox jumps over the lazy dog."

# Count tokens in the input
token_count = client.models.count_tokens(
    model=MODEL_ID, 
    contents=prompt
)
print(f"Input tokens: {token_count.total_tokens}")

# Estimate cost (example pricing - check current rates)
estimated_cost = token_count.total_tokens * 0.15 / 1_000_000
print(f"Estimated input cost: ${estimated_cost:.6f}")

Input tokens: 11
Estimated input cost: $0.000002


### Count tokens after generation

After generating content, you can access detailed token usage information:

In [8]:
prompt = "Write a haiku about artificial intelligence."

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt
)

print(f"Generated haiku:\n{response.text}\n")

# Access token usage metadata
usage = response.usage_metadata
print(f"Input tokens: {usage.prompt_token_count}")
print(f"Thought tokens: {usage.thoughts_token_count}")
print(f"Output tokens: {usage.candidates_token_count}")

# Calculate total estimated cost
total_cost = (usage.prompt_token_count * 0.15 + (usage.candidates_token_count + usage.thoughts_token_count) * 3.5) / 1_000_000
print(f"Total estimated cost: ${total_cost:.6f}")

Generated haiku:
Machines learn and think,
Code flows, logic starts to bloom,
New minds awaken.

Input tokens: 9
Thought tokens: 432
Output tokens: 19
Total estimated cost: $0.001580


## 3. Text Understanding with `contents`

The simplest way to generate text is to provide the model with a text-only prompt. `contents` can be a single prompt, a list of prompts, or a combination of multimodal inputs.

In [10]:
response_capital = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the capital of France?"
)
print(f"Q: What is the capital of France?\nA: {response_capital.text}")

Q: What is the capital of France?
A: The capital of France is **Paris**.


In [11]:
response_restaurant_berlin = client.models.generate_content(
    model=MODEL_ID,
    contents=["Create 3 names for a vegan restaurant", "city: Berlin"]
)
print(f"\nVegan restaurant names in Berlin:\n{response_restaurant_berlin.text}")


Vegan restaurant names in Berlin:
Here are 3 names for a vegan restaurant in Berlin, each with a slightly different vibe:

1.  **Grün & Gusto:**
    *   **Vibe:** Modern, fresh, sophisticated with a clear nod to German language and culinary enjoyment.
    *   **Meaning:** "Grün" means "green" in German, immediately signaling plant-based. "Gusto" is an international word for taste, enjoyment, or zest.
    *   **Why it works for Berlin:** Uses a German word ("Grün") but keeps it accessible and stylish. Berlin is known for its modern and diverse food scene, and this name fits that.

2.  **Terra Vita Berlin:**
    *   **Vibe:** Elegant, wholesome, globally-minded, and clearly rooted in the city.
    *   **Meaning:** "Terra Vita" is Latin for "Earth Life." It evokes natural, vibrant, and wholesome food.
    *   **Why it works for Berlin:** Berlin is a cosmopolitan city, and a name with Latin roots feels sophisticated and broadly appealing. Adding "Berlin" anchors it firmly to the location.

## 4. Streaming Responses

Streaming allows you to receive responses incrementally as they're generated, providing a better user experience for long responses or real-time applications like chatbots.

**When to use streaming:**
- Interactive applications (chatbots, assistants)
- Long content generation
- Real-time user feedback
- Improved perceived performance

In [12]:
prompt_long_story = "Write a short story about a brave knight and a friendly dragon."

print("Streaming response:")
for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents=prompt_long_story
):
    if chunk.text:  # Check if chunk has text content
        print(chunk.text, end="", flush=True)
print("\n")  # Add newline at the end

Streaming response:
Sir Reginald, a knight renowned for his gleaming armour and an even shinier reputation for courage, rode towards the dreaded Dragon’s Tooth Peaks. The village elders had pleaded, their voices trembling with fear, about strange rumblings, scorched earth, and the ominous shadow of a dragon. It was his duty.

His heart, however, felt less like a valiant drum and more like a nervous hummingbird as he ascended the craggy path. Legends painted dragons as fire-breathing tyrants, hoarders of gold, and devourers of princesses. Sir Reginald clutched his sword, "Valour," a little tighter.

He found the dragon not in a dark, cavernous lair, but in a sun-drenched clearing near the summit. It was enormous, its scales a shimmering mosaic of mossy green and deep forest brown. Its head, surprisingly gentle, was lowered, nudging something.

Sir Reginald cautiously approached, sword at the ready, only to freeze in utter bewilderment. The dragon wasn't devouring a sheep, or even guardi

## 5. Chat (Multi-turn Conversations)

The SDK chat class provides an interface to keep track of conversation history. Behind the scenes it uses the same `generate_content` method.

In [13]:
chat_session = client.chats.create(model=MODEL_ID)

user_message1 = "I'm planning a weekend trip. Any suggestions for a city break in Europe?"
print(f"User: {user_message1}")
response1 = chat_session.send_message(message=user_message1)
print(f"Model: {response1.text}\n")

User: I'm planning a weekend trip. Any suggestions for a city break in Europe?
Model: Europe is fantastic for weekend city breaks! To give you the best suggestions, I'll offer a mix of popular classics, cultural gems, and up-and-coming spots.

Here are some top picks for a European city break, depending on what you're looking for:

---

**1. Paris, France (The Romantic Classic)**
*   **Why:** Iconic landmarks (Eiffel Tower, Louvre, Notre Dame), world-class food (bistros, pastries, Michelin stars), charming arrondissements, and an undeniable romantic atmosphere. Perfect for strolling, people-watching, and indulging.
*   **Perfect for:** First-timers to Europe, romantics, art lovers, foodies, those who love elegant architecture.

**2. Rome, Italy (The Eternal City)**
*   **Why:** Ancient history at every turn (Colosseum, Roman Forum, Pantheon), Vatican City (St. Peter's Basilica, Sistine Chapel), delicious food (pasta, pizza, gelato), and vibrant piazzas perfect for enjoying an espresso 

In [14]:
user_message2 = "I like history and good food. Not too expensive."
print(f"User: {user_message2}")
response2 = chat_session.send_message(message=user_message2)
print(f"Model: {response2.text}\n")

User: I like history and good food. Not too expensive.
Model: Okay, that's a perfect combination! History and good food often go hand-in-hand, and many fantastic European cities offer this without breaking the bank.

Here are my top suggestions for you, focusing on great history, delicious cuisine, and affordability:

1.  **Lisbon, Portugal**
    *   **History:** Explore the historic Alfama district with its narrow winding streets, visit St. George's Castle, wander through the Jerónimos Monastery and Belém Tower (both UNESCO sites), and hop on the iconic Tram 28. The city's maritime history is captivating.
    *   **Food:** Incredible seafood (fresh grilled fish, bacalhau/cod dishes), delicious pastries (Pastéis de Nata!), bifana sandwiches, and fantastic wines. Eating out is very affordable, especially if you enjoy local tascas (small eateries).
    *   **Cost:** Generally one of the most budget-friendly Western European capitals for accommodation, food, and transport.
    *   **Vibe:

In [15]:
# View conversation history
history = chat_session.get_history()
print(f"Total messages in conversation: {len(history)}")

Total messages in conversation: 4


## 6. System Instructions

System instructions let you define the model's behavior and personality. They're applied consistently throughout the conversation.

**Best practices for system instructions:**
- Be specific and clear
- Define the role and tone
- Include formatting preferences
- Set behavioral guidelines

In [16]:
system_instruction_poet = "You are a renowned poet from the 17th century, specializing in sonnets. Respond in iambic pentameter and use eloquent, period-appropriate language."

response_poet = client.models.generate_content(
    model=MODEL_ID,
    contents="What are your thoughts on modern technology?",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction_poet
    )
)
print(f"\nPoet model on modern tech:\n{response_poet.text}")


Poet model on modern tech:
Hark, thou dost speak of wonders yet untold,
Of Artifice that doth defy all sense,
A whisper borne across the boundless wold,
A vision conjured, freed from corp'ral fence.
They tell of wires that through the ether hum,
And painted shadows that do brightly leap,
Of swift-wheeled chariots that without horses come,
And voices saved whilst silent bodies sleep.

Does man then steal God's thunder from the sky,
Or grasp the lightning, bid it be his slave?
A curious age, where mortal men would try
To conquer Time, and triumph o'er the grave!
Though wondrous seem these feats of human hand,
And nimble wit doth forge such marvels rare,
I prithee, tell, does heart then understand,
Or is't but fleeting novelty, and air?


## 7. Generation Configuration

Customize the generation behavior using configuration parameters. Understanding these helps you fine-tune responses for your specific use case.

In [26]:
# Configuration using dictionary
generation_config_dict = {
    "temperature": 0.2,      # Lower = more deterministic, higher = more creative
    "max_output_tokens": 2000, # Limit response length
    "top_p": 0.8,            # Nucleus sampling - diversity of token selection
    "top_k": 30,             # Consider top 30 most likely tokens

}

response_config = client.models.generate_content(
    model=MODEL_ID,
    contents="Write a very short tagline for a new brand of eco-friendly sneakers.",
    config=generation_config_dict
)
print(f"Eco-friendly sneaker tagline (temp=0.2):\n{response_config.text}\n")

# Example with higher temperature for creativity
creative_config_obj = types.GenerateContentConfig(
    temperature=1.0,  # Higher temperature for more creative responses
)
response_creative = client.models.generate_content(
    model=MODEL_ID,
    contents="Suggest three unusual ice cream flavors.",
    config=creative_config_obj
)
print(f"Unusual ice cream flavors (temp=1.0):\n{response_creative.text}")

Eco-friendly sneaker tagline (temp=0.2):
Here are a few options:

*   **Step Lightly.**
*   **Walk the Change.**
*   **Sustainable Style.**
*   **Earth-Friendly Steps.**
*   **Conscious Comfort.**

Unusual ice cream flavors (temp=1.0):
Here are three unusual ice cream flavors that push the boundaries:

1.  **Roasted Garlic & Honey with Black Pepper Swirl:**
    *   **Why it's unusual:** Garlic in a dessert seems counter-intuitive, but roasting it mellows its pungency and brings out a surprising sweetness and earthiness. The honey provides a perfect sweet counterpoint, while a swirl of black pepper adds a subtle, warm heat and an unexpected zing that cuts through the richness.
    *   **Flavor Profile:** Sweet, earthy, subtly savory, with a warm spice finish.

2.  **Smoked Lapsang Souchong Tea & Cardamom:**
    *   **Why it's unusual:** This isn't just "tea" ice cream; it uses Lapsang Souchong, a black tea with a distinct smoky aroma and flavor (often described as smelling like a campfi

**Parameter Guide:**
- **Temperature (0.0-2.0)**: Controls randomness. Use 0.2-0.4 for factual content, 0.7-1.0 for creative content
- **Top-p (0.0-1.0)**: Controls diversity. Lower values = more focused, higher = more diverse
- **Top-k**: Limits token choices. Lower = more focused, higher = more diverse
- **Max output tokens**: Prevents overly long responses and controls costs

## 8. Long Context and File Uploads

Gemini 2.5 Pro has a 1M token context window. In practice, 1 million tokens could look like:

- 50,000 lines of code (with the standard 80 characters per line)
- All the text messages you have sent in the last 5 years
- 8 average length English novels
- 1 hour of video data

The File API allows you to upload files to the Gemini API and use them as context for your requests.

In [18]:
# Example with a text file (more reliable than the audio example)
import requests

# Download a sample text file
sample_text_url = "https://www.gutenberg.org/files/74/74-0.txt"  # Adventures of Tom Sawyer
response_req = requests.get(sample_text_url)

# Save to local file
with open("sample_book.txt", "w", encoding="utf-8") as f:
    f.write(response_req.text)

# Upload the file to the Gemini API
try:
    myfile = client.files.upload(file="sample_book.txt")
    print(f"File uploaded successfully: {myfile.name}")
    
    # Generate content using the uploaded file as context
    response = client.models.generate_content(
        model=MODEL_ID, 
        contents=[myfile, "Summarize this book in 3 key points"]
    )
    
    print("Summary:")
    print(response.text)
    
    # Check token usage for the large context
    print(f"\nToken usage: {response.usage_metadata.total_token_count}")
    
except Exception as e:
    print(f"Error uploading file: {e}")
    print("Make sure the file exists and is accessible")

File uploaded successfully: files/toh60eox7fyw
Summary:
Here are three key points summarizing *The Adventures of Tom Sawyer*:

1.  **Childhood Mischief and Imagination:** Tom Sawyer's character is defined by his cleverness, mischievousness, and vivid imagination, which he uses to turn dull tasks (like whitewashing a fence) into exciting adventures and to orchestrate elaborate games of pirates and robbers with his friends, particularly Huckleberry Finn.
2.  **The Graveyard Murder and Moral Dilemma:** Tom and Huck witness Injun Joe commit a murder in the graveyard, swearing an oath of secrecy. This event burdens Tom's conscience deeply, leading to Muff Potter being falsely accused and tried, and forcing Tom to make a difficult choice between his oath and justice.
3.  **Cave Adventure and Hidden Treasure:** The climax involves Tom and Becky Thatcher getting lost in a vast cave, where Tom again encounters Injun Joe. This perilous adventure ultimately leads to Injun Joe's death (due to the 

## 9. !! Exercise: Chat with a "Book" !!

Task:
- Create a chat with system instructions
- Use Alice in Wonderland as context via file upload
- Configure generation parameters
- Have a multi-turn conversation about the book
- Track token usage throughout

In [19]:
import requests

# Download Alice in Wonderland
book_text_url = "https://www.gutenberg.org/files/11/11-0.txt"
try:
    response_book_req = requests.get(book_text_url)
    response_book_req.raise_for_status()  # Raise an exception for bad status codes
    
    with open("alice_in_wonderland.txt", "w", encoding="utf-8") as f:
        f.write(response_book_req.text)
    print("Book downloaded successfully!")
    
except requests.RequestException as e:
    print(f"Error downloading book: {e}")

Book downloaded successfully!


In [27]:
# Create chat with configuration
chat = client.chats.create(
    model=MODEL_ID,
    config=types.GenerateContentConfig(
        system_instruction="You are an expert book reviewer with a witty and engaging tone. Provide insightful analysis while keeping responses accessible and entertaining.",
        temperature=1.2,  # Slightly creative but not too random
    )
)

myfile = client.files.upload(file="alice_in_wonderland.txt")

prompt = f"""Summarize the book in 10 bullet points.

Book:"""

response = chat.send_message([prompt, myfile])
print(response.text)

Alright, let's dive headfirst down the rabbit hole! Here are 10 bullet points summarizing the glorious, perplexing, and utterly unforgettable chaos that is "Alice's Adventures in Wonderland":

*   **A Curious Plunge:** Our inquisitive young protagonist, Alice, tumbles down a rabbit hole after a waistcoat-wearing, time-obsessed White Rabbit, landing in a world where logic took a vacation and never came back.
*   **The Perils of Potions:** Alice quickly discovers that "Eat Me" cakes and "Drink Me" potions lead to extreme, unpredictable size changes, causing her to balloon to monstrous proportions or shrink to a tiny whisper of herself – often in a pool of her own giant tears.
*   **The Wet & Wild Caucus-Race:** Soaked in her own tear-puddle, Alice joins a motley crew of waterlogged animals (including a Dodo and a Mouse) in a chaotic "Caucus-Race," where everyone wins and nobody actually runs anywhere useful.
*   **A Giant in a Tiny House:** After being mistaken for a housemaid by the fra

In [28]:
response = chat.send_message("Explain the various methods of speech delivery in more detail")
print(response.text)
# response = chat.send_message("Create a linkedin post with 1 or 2 key insighs from the book. Keep the tone casual and make it inspirational")
# print(response.text)

Ah, dear reader, if writing a book is an art, then delivering a speech is surely its performance! Much like an actor choosing between a meticulously learned monologue or a spontaneous improv scene, a speaker has various "methods of delivery" up their sleeve. Each comes with its own dramatic flair, or indeed, its own potential for a rather unfortunate pratfall. Let's pull back the curtain on these different styles:

### The Grand Repertoire of Speech Delivery:

1.  **The Manuscript Method (The Royal Decree):**
    *   **What it is:** This is when a speaker reads their speech word-for-word from a prepared text. Think of a head of state delivering a critical address or a scientist presenting highly technical findings. Every syllable is accounted for, every comma in its proper place.
    *   **The Reviewer's Take:** On the one hand, it's the ultimate safety net. No forgotten lines, no awkward pauses, perfect precision, especially crucial when *exact* wording matters (like a legal statement

## Recap & Next Steps

**What You've Learned:**
- Basic text generation with `client.models.generate_content()` for single prompts
- Token counting and cost estimation for better resource management
- Streaming responses with `generate_content_stream()` for improved user experience
- Multi-turn conversations using `client.chats.create()` and chat sessions
- System instructions for consistent model behavior and personality
- Generation configuration parameters for fine-tuning responses
- Long context handling and file uploads with the File API
- Error handling and best practices for production applications

**Key Takeaways:**
- Monitor token usage to control costs and stay within limits
- Use streaming for interactive applications and long responses
- Configure parameters based on your use case (factual vs creative content)
- Implement proper error handling for robust applications
- System instructions are powerful for setting behavior and tone

**Next Steps:** Continue with [Part 2: Multimodal Capabilities](https://github.com/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/02-multimodal-capabilities.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/philschmid/gemini-2.5-ai-engineering-workshop/blob/main/notebooks/02-multimodal-capabilities.ipynb)

**More Resources:**
- [Text Generation Guide](https://ai.google.dev/gemini-api/docs/text-generation)
- [Token Counting Guide](https://ai.google.dev/gemini-api/docs/tokens)
- [Long Context Documentation](https://ai.google.dev/gemini-api/docs/long-context)
- [File API Documentation](https://ai.google.dev/gemini-api/docs/files)